In [1]:
import torch #main torch module
import torch.nn as nn #neural net module
import torch.optim as optim #optimizers
import torch.nn.functional as F #functions like ReLu Sig Tanh etc
from torch.utils.data import DataLoader #help us with datasets
import torchvision
import torchvision.datasets as datasets #using to access std data
import torchvision.transforms as transforms #transformations

from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(str(device) + " is being used")

cuda is being used


### Network Definitions

Below we will import a VGG16 Model

In [3]:
# Hyperparameters
input_size = 784
in_channels = 3
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 20

In [4]:
#creating an identity class that lets an input flow as is
class Identity(nn.Module):
    def __init__(self):
        super(Identity,self).__init__()
    
    def forward(self, x):
        return x

In [5]:
# Load the model, pretrained
model = torchvision.models.vgg16(pretrained=True)

In [6]:
#we need to make it so that earlier layers dont get trained
# then we dont train initial layers and only train last layers
# new layers we add will be trainable
for param in model.parameters():
    param.requires_grad = False

# We need to erase the average pooling
# classifier must be 10 features

model.avgpool = Identity()

In [7]:
#model.classifier = nn.Linear(512, 10)
#maybe we can use sequential: (multiple layers can be added in sequence)
model.classifier = nn.Sequential(nn.Linear(512, 100),
                                 nn.Dropout(0.30),
                                 nn.Linear(100, 10))

In [8]:
#init the model
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

### Model Training & Evaluation

In [9]:
# Loading Test Data
root_dirpath = '/media/ishaanaditya/2CCA1EC4CA1E8A6A/Datasets/'

test_dataset = datasets.CIFAR10(root=root_dirpath, train=False,  transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

# Loading Data
train_dataset = datasets.CIFAR10(root=root_dirpath,train=True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size,shuffle = True)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
%%time
# training loop
for epoch in range(num_epochs):
    
    #start of progress bar loop
    loop = tqdm(enumerate(train_loader), leave = False, total = len(train_loader))
    #tqdm leave param makes in one progress bar
    
    for batch_idx,(data, targets) in loop:
        # data to devices
        data = data.to(device)
        targets = targets.to(device)
        #data = data.reshape(data.shape[0],-1)#flattens
        # this is flatten because cnn accepts
        #fwd
        scores = model(data)
        loss = criterion(scores, targets)
        
        #back
        optimizer.zero_grad() #so that it does not store prev backprop calc
        loss.backward()
    
        #gradient desc
        optimizer.step()
        
        loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
        loop.set_postfix(loss = loss.item())

CPU times: user 5min 39s, sys: 4.72 s, total: 5min 44s
Wall time: 5min 31s


### Accuracy Evaluation

In [ ]:
# Checking Performance
from helper_script import check_accuracy_CNN
check_accuracy_CNN(train_loader,model)
check_accuracy_CNN(test_loader,model)